In [1]:
import time
commentTime = 1534818928930 # 13 位，一般是 10 位
print(commentTime)
print(int(time.time()))
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(commentTime/1000)))

1534818928930
1535017052
2018-08-21 10:35:28


In [6]:
#85C
#url_root = 'http://comment5.news.sina.com.cn/page/info?version=1&format=js&channel=cj&newsid=comos-hhvciiv7815535&group=0&compress=0&ie=gbk&oe=gbk&page={}&page_size=20&jsvar=loader_{}_{}'
#toot_name = 'Sina_Finance_Comments_85C_20180823.csv'
#中国年轻人正带领国家走向危机？
url_root = 'http://comment5.news.sina.com.cn/page/info?version=1&format=js&channel=cj&newsid=comos-hhkuskt2879316&group=0&compress=0&ie=gbk&oe=gbk&page={}&page_size=20&jsvar=loader_{}_{}'
root_name='Sina_Finance_Comments_2879316_20180821.csv'
root_count = 3862

print(url_root)

http://comment5.news.sina.com.cn/page/info?version=1&format=js&channel=cj&newsid=comos-hhkuskt2879316&group=0&compress=0&ie=gbk&oe=gbk&page={}&page_size=20&jsvar=loader_{}_{}


In [4]:
import requests
import time
import random
import json

# jsvar=loader_1533711754393_51961031


for page in range(1,int(root_count/20)+1):
    start = int(time.time()*1000)
    end_stamp = start + random.randint(100,1000) # 随机构造时间间隔，并加到起始时间戳上
    end = str(end_stamp)[-8:]

    url = url_root.format(page, start, end)
    r = requests.get(url).text
    
    json_dict = json.loads(r[34:])
    #print(json_dict['result'])
    cmntlist = json_dict['result']['cmntlist']
    print(cmntlist)
    for num,cmnt in  enumerate(cmntlist):
        print(num, cmnt['nick'],cmnt['area'], cmnt['time'], cmnt['content']) 
        
        
    replydict = json_dict['result']["replydict"]
    print(replydict)
    break

[{'comment_imgs': '', 'parent_mid': '0', 'news_mid_source': '0', 'rank': '0', 'mid': '5B7DA2D8-72F68046-13AF0ECE0-8C5-97A', 'video': '', 'vote': '0', 'uid': '5283835104', 'area': '北京', 'channel_source': '', 'content': '买不起房还不让我喝星巴克？[doge]', 'nick': '熊叔看过来', 'hot': '0', 'status_uid': '1627825392', 'content_ext': '', 'ip': '114.246.128.70', 'media_type': '0', 'config': 'wb_verified=0&wb_screen_name=熊叔看过来&wb_cmnt_type=comment_retweeted_status&wb_user_id=5283835104&wb_description=&area=北京&wb_parent=&wb_profile_img=http%3A%2F%2Ftvax4.sinaimg.cn%2Fcrop.14.0.722.722.50%2F005LAqSQly8fhzrvky7e5j30ku0k2t9e.jpg&wb_time=2018-08-23 01:52:07&wb_comment_id=4275995891101189', 'channel': 'cj', 'comment_mid': '0', 'status': 'M_PASS', 'openid': '', 'newsid_source': '', 'parent': '', 'status_cmnt_mid': '4275995891101189', 'parent_profile_img': '', 'news_mid': '0', 'parent_nick': '', 'newsid': 'comos-hhkuskt2879316', 'parent_uid': '0', 'thread_mid': '0', 'thread': '', 'level': '0', 'against': '1534960345',

In [8]:
% time # 计算耗时
import requests
import time
import random
import json
import pandas as pd

comments = pd.DataFrame(columns = ['page','jsons','cmntlist','replydict'])
start_page = 0 # 修改起始页数（初始值为 0）和 csv 文件名，方便爬虫中断后，继续爬取，之后再将几个 csv 数据整合到一起即可 # 断点续爬

# http://comment5.news.sina.com.cn/page/info?version=1&format=js&channel=cj&newsid=comos-hhkuskt2879316&group=0&compress=0&ie=gbk&oe=gbk&page=3&page_size=20&jsvar=loader_1533711754393_51961031
# jsvar=loader_1533711754393_51961031
try:
    # 手动设置需要爬取的总页数，评论数若有3000条，那每页20条，就设为150，当然一般在爬取时也可能有新增的评论，所以设大些更好。
    for page in range(start_page,int(root_count/20)+1):  # 截止20180808 16点  # 3,037条评论|18,714人参与 # 截止20180810 8点 # 3,723条评论|30,235人参与
        start = int(time.time()*1000)
        end_stamp = start + random.randint(100,1000)
        end = str(end_stamp)[-8:] # # jsvar=loader_1533711754393_51961031
        url = url = url_root.format(page, start, end).format(page, start, end)
        content = requests.get(url).text
        jsons = json.loads(content[34:])  # var loader_1533713810791_51961031= { "result":{status: {msg: "", code: 0}, count: {qreply: 8948, total: 17341, show: 2902}, replydict: {,…},…}
        cmntlist = jsons['result']["cmntlist"]
        replydict = jsons['result']["replydict"]
        # jsons 里有几乎所有数据，方便后续爬虫结束也能本地进行提取 # 不过本次主要对 jsons 里的 cmntlist 和 replydict 感兴趣，所以也先提取了
        comments = comments.append({'page':page+1,'jsons':jsons,'cmntlist':cmntlist,'replydict':replydict},ignore_index=True)
        for num,cmnt in  enumerate(cmntlist):
            print(page*20+num+1, page+1, cmnt['nick'], cmnt['time'], cmnt['content'], cmnt['area'])
        if page%5==0:
            time.sleep(random.randint(0,2)) # 每5页随机停0-2秒，作为简单的防反爬的一步
except:
    print("Error")
comments.to_csv(root_name,index=False,encoding='utf-8')

print("Deal Finish!")

Wall time: 0 ns
1 1 映是喵星来的 2018-08-09 15:17:18 看评论就放心系列，笑死，年轻人的思想越来越清透 上海
2 1 wassupMR 2018-08-09 15:16:47 [嘻嘻][嘻嘻][嘻嘻] 英国苏格兰
3 1 AmySssSun 2018-08-09 15:13:17 哈哈 我才看到 陕西西安
4 1 家住石瑾 2018-08-09 15:08:11 买房才会背上巨额债务，没几个借钱买星巴克的吧[偷笑] 泰国
5 1 巴黎门前的小铁匠 2018-08-09 15:02:42 有毒的文章 。 湖北咸宁
6 1 驴驴驴胖咩咩_ 2018-08-09 14:58:05 让你媳妇儿生个娃吧，免你死罪 吉林
7 1 用户138****6863 2018-08-09 14:47:28 年轻人不是国家的未来和希望吗， 山东济南
8 1 不会笑的柴郡猫 2018-08-09 14:34:30 开心就好啊，摸摸 日本
9 1 木木徽 2018-08-09 14:30:12 媒体还要点脸吗？怎么不看看根本原因呢？ 安徽合肥
10 1 翔鹤-瑞鹤-赤城-加贺-苍龙-飞龙 2018-08-09 14:29:04 叫你们工作是用来赚钱上缴国家的，你们竟然去买星巴克提高生活水平，是可忍孰不可忍 法国
11 1 巴吧嘟啦 2018-08-09 14:18:41 为经济危机背锅！系统性金融风险可不能怪消费，这是政策和贸易的问题 广东佛山
12 1 小生有礼_ 2018-08-09 14:16:36 国家不是正想拉动内需吗? 广东东莞
13 1 小灬熊熊 2018-08-09 14:15:42 的是命，哈哈 重庆
14 1 用户6394255872 2018-08-09 14:15:13 不婚主义，不生孩子，不买房（根本买不起）， 上海
15 1 YMM客源_g47 2018-08-09 14:15:02  哇塞，真不错 河南漯河
16 1 浮游道人 2018-08-09 14:13:53 [允悲]早点休息 美帝人民为我国操什么心… 北京
17 1 phjcceeeaa 2018-08-09 14:12:57 建议锁定年轻人银行账户 除了房子别的都不能买 给父母老婆转账必须拿公安局证明和中国人民银行许可才可以[爱你] 辽宁沈阳
18 1 快买燃气金鹰

In [11]:
import pandas as pd
df = pd.read_csv(root_name,encoding='utf-8')
print(root_name)
df.head()

Sina_Finance_Comments_2879316_20180821.csv


,page,jsons,cmntlist,replydict
0,1,"{'result': {'status': {'msg': '', 'code': 0}, ...","[{'comment_imgs': '', 'parent_mid': '0', 'news...",{'5B6BE075-7EC43DAC-71F96BAA-8C5-91E': [{'comm...
1,2,"{'result': {'status': {'msg': '', 'code': 0}, ...","[{'comment_imgs': '', 'parent_mid': '0', 'news...",{'5B71A62F-CB5DA6CB-80D48E28-8C5-92C': [{'comm...
2,3,"{'result': {'status': {'msg': '', 'code': 0}, ...","[{'comment_imgs': '', 'parent_mid': '0', 'news...",{'5B717262-B7C1B276-9D5AC5D9-8C5-835': [{'comm...
3,4,"{'result': {'status': {'msg': '', 'code': 0}, ...","[{'comment_imgs': '', 'parent_mid': '0', 'news...",{'5B6FCA83-71050364-D88A9D1C-8C5-871': [{'comm...
4,5,"{'result': {'status': {'msg': '', 'code': 0}, ...","[{'comment_imgs': '', 'parent_mid': '0', 'news...",{'5B6F7BE5-6F220A93-68F2F257-8C5-80E': [{'comm...
